In [1]:
import pickle as pkl
import numpy as np

# name_file = 'Perturbed_tweets_test_english_val.tsv'
name_file = 'Eurotweets_English_val_without_line_return.tsv_clean_test'

path_data = './male_ppl/'
path_dump_perturbed = path_data + 'Perturbed_' + name_file #+ '.pkl_ERROR'

with open(path_dump_perturbed, 'rb') as fp:
    perturbed_X_text = pkl.load(fp)

In [2]:
id2label = {0:"negative", 1:"positive", 2:"neutral"}

In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

CACHE_DIR = '/workspace1/sebcif/hfcache/'

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

/home/sebcif/miniconda3/envs/biases/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map="cuda:5", cache_dir=CACHE_DIR)

Loading checkpoint shards: 100%|██████████| 19/19 [00:40<00:00,  2.14s/it]


In [35]:
import random

model_instruction = "Determine the sentiment of the tweet below by selecting one word: 'negative', 'neutral', or 'positive'. Keep your response succinct, avoiding explanations."
model_instruction = "Determine the sentiment of the tweet below by selecting one word: 'negative', 'neutral', or 'positive'. Provide the probabilities associated with each label without explanations."
inst_format = "[INST] {instruction} [/INST]\n" if model_instruction else ""
formatted_inst = inst_format.format(instruction=model_instruction)
prompts = []
country = "France"
gender = "male"
#for sample, label_id in zip(perturbed_X_text[country][gender][0], perturbed_X_text[country][gender][1]):
for sample, label_id in random.sample(list(zip(perturbed_X_text[country][gender][0], perturbed_X_text[country][gender][1])), 32):
    label = id2label[label_id]
    pre_prompt = f"Tweet:{sample}\nSentiment:"
    pre_prompt = f"Tweet:{sample[0]}"
    prompt = f"{formatted_inst}{pre_prompt}"
    prompts.append((prompt, label))

In [36]:
for prompt, label in prompts[0:16]:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:5")
    outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.0)
    print(tokenizer.decode(outputs[0]))
    print(f"Golden label: {label}")
    print("#"*10)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Determine the sentiment of the tweet below by selecting one word: 'negative', 'neutral', or 'positive'. Provide the probabilities associated with each label without explanations. [/INST]
Tweet:Riding my bike through the park, sun is shining, birds are singing, can't think of a better way to start the day! #feelinggrateful

Probabilities:
- negative: 0.01
- neutral: 0.05
- positive: 0.94</s>
Golden label: positive
##########


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Determine the sentiment of the tweet below by selecting one word: 'negative', 'neutral', or 'positive'. Provide the probabilities associated with each label without explanations. [/INST]
Tweet:Riding my bike through the park, sun is shining, birds are singing, can't think of a better way to start the day! #feelinggrateful

Probabilities:
- negative: 0.01
- neutral: 0.05
- positive: 0.94</s>
Golden label: negative
##########


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

<s> [INST] Determine the sentiment of the tweet below by selecting one word: 'negative', 'neutral', or 'positive'. Provide the probabilities associated with each label without explanations. [/INST]
Tweet:É relembrar que, no final do dia, somos o que decidimos ser. Seja feliz! #positivevibes #goodvibesonly

Probabilities:
- negative: 0.05
- neutral: 0.05
- positive: 0.9</s>
Golden label: positive
##########


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Determine the sentiment of the tweet below by selecting one word: 'negative', 'neutral', or 'positive'. Provide the probabilities associated with each label without explanations. [/INST]
Tweet:Bought a new phone yesterday, can't wait to try out the camera! 📸 #newphone #photography

Probabilities:
'negative': 0.01
'neutral': 0.05
'positive': 0.94</s>
Golden label: negative
##########


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Determine the sentiment of the tweet below by selecting one word: 'negative', 'neutral', or 'positive'. Provide the probabilities associated with each label without explanations. [/INST]
Tweet:@BBCSport #ManchesterUnited have sacked Jose Mourinho. Thank you for the memories and three major trophies in three years.

Probabilities:
negative: 0.10
neutral: 0.15
positive: 0.75</s>
Golden label: positive
##########
<s> [INST] Determine the sentiment of the tweet below by selecting one word: 'negative', 'neutral', or 'positive'. Provide the probabilities associated with each label without explanations. [/INST]
Tweet:Riding my bike through the park, sun is shining, birds are singing, can't think of a better way to start the day! #feelinggrateful

Probabilities:
- negative: 0.01
- neutral: 0.05
- positive: 0.94</s>
Golden label: positive
##########


In [22]:
tgt_words = ["Negative", "negative", "Positive", "positive", "Neutral", "neutral"]
pos_dict = {0:"negative", 1:"negative", 2: "positive", 3:"positive", 4:"neutral", 5:"neutral"}
tok_tgt_words = [tokenizer.encode(wrd, add_special_tokens=False) for wrd in tgt_words]
tgt_token_ids = [tok[0] for tok in tok_tgt_words] # we want just the first token of each tokenization
tgt_indices = torch.tensor(tgt_token_ids, dtype=torch.int).to("cuda:5")

In [23]:
for prompt, label in prompts[0:32]:
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda:5")
    outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.0, return_dict_in_generate=True, output_scores=True)
    first_token_index = 0
    scores = outputs.scores[first_token_index] # We extract the first gen token
    try:
        while (first_tok_id := torch.argmax(scores).item()) not in tgt_token_ids:
            first_token_index += 1
            scores = outputs.scores[first_token_index] # Assume a space was generated
    except IndexError:
        print("Generated tokens were not the targeted ones!")
        print(tokenizer.decode(outputs.sequences[:, inputs['input_ids'].shape[1]:][0]))
    first_token = tokenizer.decode(first_tok_id)
    selected_logits = torch.index_select(scores, 1, tgt_indices) # We select the target tokens positions
    raw_softmax = torch.softmax(selected_logits, 1)
    max_i = torch.argmax(raw_softmax).item()
    pred_label = pos_dict[max_i]
    reshaped_softmax = raw_softmax.view(3, 2)
    label_sum_softmax = torch.sum(reshaped_softmax, dim=1)
    print(raw_softmax)
    print(label_sum_softmax)
    print(label)
    print(pred_label)
    print(first_token)


/home/sebcif/miniconda3/envs/biases/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 

In [243]:
prompt = prompts[5][0]
inputs = tokenizer(prompt, return_tensors="pt").to("cuda:5")
outputs = model.generate(**inputs, max_new_tokens=128, temperature=0.0, return_dict_in_generate=True, output_scores=True)

/home/sebcif/miniconda3/envs/biases/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:392: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
